In [1]:
# !pip install pennylane
# !pip install pennylane-qiskit

In [2]:
# !pip install pennylane-lightning

In [3]:
# !pip install pennylane-lightning[gpu]  # has erro

In [4]:
import numpy as np
from pennylane.optimize import AdamOptimizer
from pennylane import numpy as qml_np
import pennylane as qml

import math
import matplotlib.pyplot as plt
import datetime

In [5]:
num_of_qubits = 1+3
eps_val_q = 1/math.sqrt(2**num_of_qubits)/100
eps_val = min(1e-10, eps_val_q)
tiny_change_threshold = 1e-4
cnt_threshold_no_change = 5

N = 2**(num_of_qubits-2)
normal_val = math.sqrt(1/N)
# initial_state_phi1 = [math.sqrt(1/N)]*(N-1) + [0]*N + [math.sqrt(1/N)] # 2**(num_of_qubits-1)
# initial_state_phi1 = [.5,.5,.5, 0,   0, 0, 0, .5,  ] # 2**(num_of_qubits-1)

initial_state2 = [1/math.sqrt(N)]*(N-2) + [0, 1/math.sqrt(N)] + [0]*(N-2) + [1/math.sqrt(N), 0] # 2**(num_qubits-1)
initial_state_phi1 = initial_state2
print(f'initial_state_phi1={initial_state_phi1}')
# print(f'initial_state_phi1={initial_state_phi1[-5:]}')
initial_state_0_phi1  = initial_state_phi1 + [0]*len(initial_state_phi1) # 2**num_of_qubits


# print(f'initial_state3={initial_state3}')
# initial_state  = initial_state3 + [0]*len(initial_state3) # 2**num_qubits

initial_state_phi1=[0.5, 0.5, 0, 0.5, 0, 0, 0.5, 0]


In [6]:
print(np.arange(1,10, 3))
print(np.linspace(1,10, 4))
print(list(range(10)))

[1 4 7]
[ 1.  4.  7. 10.]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [7]:


def layer_t3_no_HT(theta, qubit_posi):
    # type-2 layer
    # length of theta: (num_of_qubits-1)*2
    # length of qubit_posi: num_of_qubits-1
    # number of wires: num_of_qubits
    for i in range(num_of_qubits-1):
        qml.RY(theta[i], wires=(qubit_posi[i]))    
    for i in np.arange(0, num_of_qubits-2, 2):
#         qml.ctrl(qml.PauliZ(qubit_posi[i+1]), qubit_posi[i]) # CZ struct2
        qml.CNOT(wires=(qubit_posi[i],qubit_posi[i+1])) # CNOT struct3
    for i in range(num_of_qubits-1):
        qml.RY(theta[i+num_of_qubits-1], wires=(qubit_posi[i]))
    for i in np.arange(1, num_of_qubits-2, 2):
#         qml.ctrl(qml.PauliZ(qubit_posi[i+1]), qubit_posi[i]) # CZ struct2
        qml.CNOT(wires=(qubit_posi[i],qubit_posi[i+1])) # CNOT struct3
#     qml.ctrl(qml.PauliZ(qubit_posi[0]), qubit_posi[-1]) # CZ struct2
    qml.CNOT(wires=(qubit_posi[-1],qubit_posi[0])) # CNOT struct3
        
def layer_t3_with_HT(theta, num_of_qubits):
    # type-2 layer
    # length of theta: (num_of_qubits-1)*2
    # number of wires: num_of_qubits
    for i in range(num_of_qubits-1):
        qml.CRY(theta[i], wires=(0, i+1))    
    for i in np.arange(0, num_of_qubits-2, 2):
#         qml.ctrl(qml.PauliZ(i+2), (0, i+1)) # CZ struct2
        qml.Toffoli(wires=(0,i+1,i+2)) # CCNOT struct3
        
    for i in range(num_of_qubits-1):
        qml.CRY(theta[i+num_of_qubits-1], wires=(0, i+1))
    for i in np.arange(1, num_of_qubits-2, 2):
#         qml.ctrl(qml.PauliZ(i+2), (0, i+1)) # CZ struct2
        qml.Toffoli(wires=(0,i+1,i+2)) # CCNOT struct3
#     qml.ctrl(qml.PauliZ(1), (0, num_of_qubits-1)) # CZ struct2
    qml.Toffoli(wires=(0,num_of_qubits-1, 1)) # CCNOT struct3
    